## MNIST Training with MXNet and Gluon

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using MXNet and the Gluon API.



In [ ]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet
from mxnet import gluon
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = get_execution_role()

## Uploading the data

We use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value `inputs` identifies the location -- we will use this later when we start the training job.

In [ ]:
!rm minc-2500-tiny.zip
!rm -rf minc-2500-tiny
!wget https://raw.githubusercontent.com/dmlc/web-data/master/gluoncv/classification/minc-2500-tiny.zip
!unzip minc-2500-tiny.zip
!aws s3 cp --recursive minc-2500-tiny s3://datalab/data/minc-2500-tiny

In [ ]:
# s3_dir = 'file:///home/ec2-user/SageMaker/meican_datalab/minc-2500-tiny/'
s3_dir = 's3://datalab/data/minc-2500-tiny/'
inputs = {'training': s3_dir+'train', 'validation': s3_dir+'val', 'test': s3_dir+'test'}
print(inputs)

## Implement the training function

We need to provide a training script that can run on the SageMaker platform. The training scripts are essentially the same as one you would write for local training, except that you need to provide a `train` function. The `train` function will check for the validation accuracy at the end of every epoch and checkpoints the best model so far, along with the optimizer state, in the folder `/opt/ml/checkpoints` if the folder path exists, else it will skip the checkpointing. When SageMaker calls your function, it will pass in arguments that describe the training environment. Check the script below to see how this works.

The script here is an adaptation of the [Gluon MNIST example](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist.py) provided by the [Apache MXNet](https://mxnet.incubator.apache.org/) project. 

In [ ]:
!cat 'transfer_learning.py'

## Run the training script on SageMaker

The ```MXNet``` class allows us to run our training function on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. In this case we will run our training job on a single c4.xlarge instance. 

In [ ]:
m = MXNet(
    "transfer_learning.py",
    source_dir=".",
    role=role,
    instance_count=1,
    instance_type="ml.p3.2xlarge",  # "local_gpu", "ml.p3.2xlarge"
    framework_version="1.8.0",
    py_version="py37",
    hyperparameters={
        "classes": 23,
        "batch-size": 64,
        "epochs": 2,
        "learning-rate": 0.001,
        "momentum": 0.9,
        "wd": 0.0001,
        "num-gpus": 1,
        "num-workers": 8,  # 1, 8
        "model-name": "ResNet50_v2"
    },
)

After we've constructed our `MXNet` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [ ]:
m.fit(inputs)

In [ ]:
training_job_name = m.latest_training_job.name
print(training_job_name)
# m = MXNet.attach(training_job_name=training_job_name)

After training, we use the MXNet object to build and deploy an MXNetPredictor object. This creates a SageMaker endpoint that we can use to perform inference. 

This allows us to perform inference on json encoded multi-dimensional arrays. 

In [ ]:
predictor = m.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")  # "local", "ml.m5.xlarge"

# from sagemaker.mxnet.model import MXNetModel
# mxnet_model = MXNetModel(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name), role=role, entry_point='transfer_learning.py', framework_version='1.8.0', py_version='py37')
# predictor = mxnet_model.deploy(instance_type="ml.m5.xlarge", initial_instance_count=1)

Or you can try SageMaker Endpoint Serverless

In [ ]:
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig

serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=1,
)

image_uri = sagemaker.image_uris.retrieve(
    framework="mxnet",
    region=sagemaker_session.boto_region_name,
    version="1.8.0",
    py_version="py37",
    instance_type="ml.m5.large",
    image_scope="inference",
)
print(image_uri)

estimator = MXNet.attach(training_job_name=training_job_name)
serverless_predictor = estimator.deploy(serverless_inference_config=serverless_config, image_uri=image_uri, entry_point='transfer_learning.py')

We can now use this predictor to classify hand-written digits. Manually drawing into the image box loads the pixel data into a 'data' variable in this notebook, which we can then pass to the MXNet predictor. 

Fetch the first image from the test dataset and display it.

In [ ]:
import cv2
image = cv2.imread('minc-2500-tiny/test/brick/brick_001968.jpg')
print('image:', type(image), image.shape, image.dtype)

The predictor runs inference on our input data and returns the predicted digit (as a float value, so we convert to int for display).

In [ ]:
response = predictor.predict(image)
print(response)

In [ ]:
response = serverless_predictor.predict(image)
print(response)

## Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
predictor.delete_endpoint()

In [ ]:
serverless_predictor.delete_endpoint()